In [5]:
# 导入相关包
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import os
import torch
from torch import nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
import json
import yaml
import joblib
from datetime import datetime

from Models.lstm_net import lstm_net
from utils.data_process.power_data_process import power_process

In [6]:
data_str='data1'
config_path='C:/project/Electric-innovation-competition/Power_Grid_Optimization/utils/configs/power_forecasting/lstm_power_forecasting.yaml'
root_path = 'C:/project/Electric-innovation-competition/Power_Grid_Optimization'
with open(config_path) as file:
    config = yaml.safe_load(file)
scaler = joblib.load( os.path.join(root_path, config['data_config'][data_str]['scaler_save_path']))
predict_model_path = config['model_config']['predict_model_path']
dataset, train_dataloader, test_dataloader = power_process(yaml_path='C:/project/Electric-innovation-competition/Power_Grid_Optimization/utils/configs/power_forecasting/lstm_power_forecasting.yaml', data=data_str)
model = torch.load(predict_model_path)

model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
scaler = joblib.load(config['data_config'][data_str]['scaler_save_path'])
for i, (X, y) in enumerate(test_dataloader):
    X = X.to(device)
    y = y.to(device)
    y_pred = model(X)
    if i == 0:
        predicted_data = y_pred.cpu().detach().numpy()
        true_data = y.cpu().detach().numpy()
    else:
        predicted_data = np.concatenate((predicted_data, y_pred.cpu().detach().numpy()))
        true_data = np.concatenate((true_data, y.cpu().detach().numpy()))


FileNotFoundError: [Errno 2] No such file or directory: 'Data/power_data/merged_power_data1.csv'

In [ ]:
# 获取目标列索引并反归一化
target_index = dataset.target_columns_index
original_col_len = len(dataset.original_data.columns)
temp_array = np.ones((len(predicted_data), original_col_len))
predicted_data = predicted_data.reshape(predicted_data.shape[0], -1 , len(target_index))
true_data = true_data.reshape(true_data.shape[0], -1 , len(target_index))
assert predicted_data.shape == true_data.shape 
predict_time = predicted_data.shape[1]
temp_predicted_data = np.ones((len(predicted_data), predict_time, original_col_len))
temp_true_data = np.ones((len(predicted_data), predict_time, original_col_len))    
for i in range(predict_time):
    for j, index in enumerate(target_index):
        temp_predicted_data[:,i,index] = predicted_data[:,i,j]
        temp_true_data[:,i,index] = true_data[:,i,j]
    # 反归一化
    # temp_predicted_data[:,i,:] = scaler.inverse_transform(temp_predicted_data[:,i,:])
    

In [ ]:

# 计算 RMSE、MAE
rmse = math.sqrt(mean_squared_error(temp_true_data[:,:,target_index].flatten(), temp_predicted_data[:,:,target_index].flatten()))
mae = mean_absolute_error(temp_true_data[:,:,target_index].flatten(), temp_predicted_data[:,:,target_index].flatten())
print("RMSE:{}".format(round(rmse,3)))
print("MAE: {}".format(round(mae, 3)))

In [ ]:
# 绘制预测图形
show_start = 0
show_end = 10
plt.figure(facecolor='white')
for i in range(predict_time):   
    plt.subplot(predict_time, 1, i+1)
    plt.plot(temp_true_data[show_start:show_end,i,target_index], label='True Data')
    plt.plot(temp_predicted_data[show_start:show_end,i,target_index], label='Prediction')
    plt.title('time step:{}min'.format(i*15+15))
    plt.legend()
plt.show()